# Inaugural Project 

By Josefine Pedersen, Viktor Texel and Pernille Svendsen

> **Table of contents** 
> - Question 1
> - Question 2
> - Question 3
> - Question 4
> - Question 5
> - Conclusion




*Imports and set magics*:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Predefine options for all plots
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"-"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Importing our own code
from inauguralproject import HouseholdSpecializationModelClass

We are aware of the **runtime warnings** throughout the project, and we have tried to remove them by not allowing for NaN values for the parameters or by not allowing for division with zero, but this did not help. We see no indications in the code to suggest we should have invalid values. Furthermore the warnings do not appear everytime we run the code. 

# Question 1

We illustrate how the following expression changes with varying values for alpha and sigma, assuming that the choice set for the model is discrete i.e. in half hours. $$ \frac{H_F}{H_M} $$  

The expression depicts the relationship for home production between females and males. We solve question 1:

In [ ]:
# Set an instance for the class
model1 = HouseholdSpecializationModelClass()

# Define lists with the values for alpha and sigma
alpha_list = [0.25, 0.5, 0.75]
sigma_list = [0.5, 1.0, 1.5]

# Solve the model for the different parameter values
model1.set_alpha_sigma(alpha_list,sigma_list)

We know that in the model *alpha* represents how productive the female is in the household relative to the male, and *sigma* represents the elasticity of substitution. 

We conclude the following:

- For alpha = 0.5 the male and female are equally productive and sigma has no effect, since there is no gain in substituting between home production performed by the male compared to the female or opposite.
- For alpha > 0.5 the female is more productive, and for wF = wM = 1, the female will work more hours in the household compared to the male. 
- For alpha < 0.5 the female is less productive, and for wF = wM = 1, the female will work less hours in the household compared to the male.
- The higher the value of sigma, the easier it is to substitute between hours worked by the male and female in the home. Thus, if alpha is larger than 0.5 the ratio is larger the higher value sigma has. If alpha is below 0.5 the ratio is lower. This is because the couple would be able to defer more hours worked at home to the more productive member of the couple.

# Question 2

We plot the following logaritmic functions against eachother using varying values for female wage: $$ log\frac{H_F}{H_M} \\\ \\\ log\frac{w_F}{w_M}$$ 

Our model is still based upon the assumption of a discrete choice set. 

We solve question 2:

In [ ]:
# Define a new model to overwrite changes made in the previous questions
model2 = HouseholdSpecializationModelClass()

# Solve the model for a vector of female wages
model2.solve_wF_vec(discrete=True, do_print=True)

**The figure** illustrates that the ratio HF/HM is decreasing in wF/wM. It depicts the relationsship between log of the relative homeproduction and log of relative wages. The plot showcases, that when the wage ratio increases, then the home production performed by the female in the household decreases. This occurs, since it is more optimal for the female to work in the market, since she gets a relatively higher wage.

However, due to the assumption of discrete time, we see that for $w_F = 0.9$ and $w_F = 1.0$ there is no change in the ratio, as the optimal new choice for home production is changed by a smaller amount than is possible when the choice set is discrete. In the following question this minor error is cleared, when we do the same task with an assumption of continous time. 

# Question 3

Now we assume that time is continous rather than discrete. This means that there are infinitely many choice set possibilities. 

Again we plot the following logaritmic functions against each other using varying values for female wage: $$ log\frac{H_F}{H_M} \\\ \\\ log\frac{w_F}{w_M}$$ 

We solve question 3:

In [ ]:
# Define a new model to overwrite changes made in the previous questions
model3 = HouseholdSpecializationModelClass()

# Solve the model for a vector of female wages
model2.solve_wF_vec(do_print=True)

**The figure** illustrates the same results as in question 2, however due to the introduction of continous time the graph is much smoother. Furthermore, the results are more precise.  

# Question 4

In Siminski and Yetsenga (2022), *Specialization, Comparative Advantage, and the Sexual Division of Labor*, the following regression is run on time use data of couples:

$$ log\frac{H_F}{H_M} = \beta_0 + \beta_1*log\frac{w_F}{w_M}$$ 

They estimate the beta values to be approximately.: $$ \beta_0 = 0.4 $$ $$ \beta_1 = -0.1 $$

That $\beta_0 = 0.4$ means that even when men and women have the same salary, so $log\left(\frac{w_f}{w_m}\right)=log(1)=0$, the ratio between men's and women's household work will still be positive, so women work more than men at home.

That $\beta_1=-0.1$ means that there is a negative correlation, so when women's wages rise relative to men's, women's household work will decrease relative to men's. So when the ratio between wages increases by 1 per cent, the ratio between household work will decrease by 0.1 per cent. However, the decrease in household work is of a relatively small magnitude.

We estimate values of alpha and sigma that gets us closest to the values found in Siminski and Yetsenga (2022). We do this by optimizing over different values of sigma and alpha until it reaches the values that minimize the following expression:

 $$ (\beta - \hat{\beta}_{0})^2 + (\beta_1 - \hat{\beta}_{1})^2 $$

In [ ]:
# Define a new model to overwrite changes made in the previous questions
model4 = HouseholdSpecializationModelClass()

# Optimize over alpha and sigma to find the best fit to data
model4.fit_data()

We check how the model matches the results from Siminski and Yetsenga (2022), when we set alpha and sigma as the values we find above:

In [ ]:
model4.par.alpha=model4.sol.optimal_alpha
model4.par.sigma=model4.sol.optimal_sigma
sol = model4.run_regression()
print(f"beta0 = {model4.sol.beta0:.2f}, beta 1 = {model4.sol.beta1:.2f}")

Illustrating the estimated model against the results from Siminski and Yetsenga (2022)

In [ ]:
# Creating linspace for x-axis
x = np.linspace(-2, 2, 100)

# Regression from Siminski and Yetsenga (2022)
y = 0.4 - 0.1 * x
y_original = model2.sol.beta0 + model2.sol.beta1 * x

# Plotting the two regressions and adding label and legend
plt.plot(x, y, label='Siminski and Yetsenga')
plt.plot(x, y_original, label='Estimated model')
plt.xlabel('log(Wf/WM)')
plt.ylabel('log(HF/HM)')
plt.title('Comparison')
plt.legend()
plt.show()

We can see that when we set alpha to the value 0.98 and sigma to the value 0.1, the optimal choices for the couple produces vectors of ratios between the home production and wages, that matches the results found by Siminski and Yetsenga (2022). 

This means that the female is very productive at home production relative to the male, and that the elasticity of substitution is relatively low. This makes sense, since data showed that even with equal wages the female would work more at home, which suggests the female should be more productive. Further, the ratio between hours worked at home changed relatively slowly in the data when the wage ratio increased, suggesting that there is a low elaticity of substitution.

# Question 5

As an extension to the model, we add the possibility for the couple to have different preferences for who should work at home, which is independent of the relative wage. 

We thus introduce the parameter k, which is positive if the couple prefers that the woman works at home, whereas the expression below becomes positive if the preferences are met. k is negative if the couple prefers that the man works at home, whereas the expression below becomes positive if the preferences are met. This ensures higher utility when the preferences for the couple is met. If the couple is indifferent in their preferences, then the expression will be zero and the model will stay as it was. 

The extension to the model is:

$$ +k*\left(\frac{HF}{HM}-\frac{HM}{HF}\right) $$

In [ ]:
# Define a new model to overwrite changes made in the previous questions
model5 = HouseholdSpecializationModelClass()

# Store the results of the optimization to find the value of k to maximize utility in the extended model
result = model5.optimal_k()

Without the extension we had the following estimates for $\beta_0$ and $\beta_1$ for the given parameters (where $\alpha=0.5$):

In [ ]:
result = model5.run_regression()
print(f'beta0: {result[0]:.2f} \nbeta1: {result[1]:.2f}')

We thereby see that the extension was only somewhat able to improve the model, since the optimal results yield a $\beta_0$ somewhat closer to the results from Siminski and Yetsenga (2022), but the value for $\beta_1$ is relatively far from the results from the article.

The optimal value for k is set as 0.00298, so it is positive meaning that the couple would have a preference for the female to work more hours at home, which fits with the data, since the constant value was positive.

It makes sense that we cannot fit the data perfectly, since our baseline model had to have $\alpha≈1$ for the model to fit the data.

# Conclusion

To sum up, we started by looking at a discrete choice set and found that a higher value of sigma makes substitution easier between hours worked at home by females and males. 
Together with alpha, which represents the productivity of females in the household relative to males, we found that for alpha = 0.5, sigma has no effect as both are equally productive. For higher values of alpha the female is more productive and for lower values the female is less productive. 

Furthermore, we found a decreasing relationship between relative homeproduction and relative wages. This means that when female wages increases relative to males the homeroduction performed by the female decreases. Due to discrete time we see no change when female wages is 0.9 and 1.0. We adjust this result by introducing continuous time allowing for infinitely many choice sets.
By introducing continuous time we still find a negative relationship between homeproduction and wages, but now with a much smoother and more precise result. 
Hereby improving the model. 

Next we look at a regression on homeproduction from Siminski and Yetsenga (2022) and estimate values of alpha and sigma that take us closest to the results for $\beta_0$ and $\beta_1$ found in the article. 
Looking at different values for alpha and sigma, we find that alpha = 0.98 and sigma = 0.1 is the closest we can get. This means that females are much more productive in the household relative to the male, with a low elasticity of substitution. 

Lastly we attempt to extend the model by introducing the couples preferences for the distribution of hours spent at home, which is independent of the relative wage. We introduce the parameter k, which is positive if the couple prefers that the woman works at home, whereas the couple gets more utility if the preferences are met. Opposite for men. If the couple is indifferent in their preferences  the model is unchanged. We where not really able to improve the model according to the results in Siminski and Yetsenga (2022).